In [125]:
from pulp import *
import pandas as pd
import numpy as np
import itertools

In [183]:
human = pd.read_csv("datasets/simple_hr/hr_data - human.csv")
section = pd.read_csv("datasets/simple_hr/hr_data - section.csv")

In [184]:
human.head()

,id,cost,skill_a,skill_b,skill_c,skill_d
0,1,5,1,1,0,0
1,2,3,0,0,1,0
2,3,4,0,1,0,1
3,4,2,1,0,0,0
4,5,5,0,0,1,1


In [185]:
section.head()

,id,num,skill_center_a,skill_center_b,skill_center_c,skill_center_d,dist
0,1,5,1,0,0,0,1
1,2,5,0,1,0,0,-1
2,3,4,0,0,1,0,1
3,4,3,0,0,0,1,1
4,5,3,0,1,0,0,-1


In [186]:
def to_uni_vec( x ):
    dist = np.sqrt(x.skill_a + x.skill_b + x.skill_c + x.skill_d)
    vec = {
        "skill_a_uni" : x.skill_a/dist, 
        "skill_b_uni" : x.skill_b/dist, 
        "skill_c_uni" : x.skill_c/dist, 
        "skill_d_uni" : x.skill_d/dist, 
    }
    return pd.Series(vec)

human = pd.concat( [human, human.apply( to_uni_vec, axis=1 )], axis=1 )
human.head()

,id,cost,skill_a,skill_b,skill_c,skill_d,skill_a_uni,skill_b_uni,skill_c_uni,skill_d_uni
0,1,5,1,1,0,0,0.707107,0.707107,0.000000,0.000000
1,2,3,0,0,1,0,0.000000,0.000000,1.000000,0.000000
2,3,4,0,1,0,1,0.000000,0.707107,0.000000,0.707107
3,4,2,1,0,0,0,1.000000,0.000000,0.000000,0.000000
4,5,5,0,0,1,1,0.000000,0.000000,0.707107,0.707107


In [187]:
m = LpProblem() # 部署ごとのコストを最小化する
# 最小化
for i, row in section.iterrows():
    human["section_%d"%row.id] = [LpVariable("v%d_%d"%(row.id,j), cat="Integer", upBound=1, lowBound=0) for j in human.index]
human.head()

,id,cost,skill_a,skill_b,skill_c,skill_d,skill_a_uni,skill_b_uni,skill_c_uni,skill_d_uni,section_1,section_2,section_3,section_4,section_5
0,1,5,1,1,0,0,0.707107,0.707107,0.000000,0.000000,v1_0,v2_0,v3_0,v4_0,v5_0
1,2,3,0,0,1,0,0.000000,0.000000,1.000000,0.000000,v1_1,v2_1,v3_1,v4_1,v5_1
2,3,4,0,1,0,1,0.000000,0.707107,0.000000,0.707107,v1_2,v2_2,v3_2,v4_2,v5_2
3,4,2,1,0,0,0,1.000000,0.000000,0.000000,0.000000,v1_3,v2_3,v3_3,v4_3,v5_3
4,5,5,0,0,1,1,0.000000,0.000000,0.707107,0.707107,v1_4,v2_4,v3_4,v4_4,v5_4


In [188]:
section["dist_v"] = [LpVariable("dist_%d"%(i)) for i in section.index]
section.head()

,id,num,skill_center_a,skill_center_b,skill_center_c,skill_center_d,dist,dist_v
0,1,5,1,0,0,0,1,dist_0
1,2,5,0,1,0,0,-1,dist_1
2,3,4,0,0,1,0,1,dist_2
3,4,3,0,0,0,1,1,dist_3
4,5,3,0,1,0,0,-1,dist_4


In [200]:
# 最小化
cost = None
for i, row in section.iterrows():
    cost += lpDot(human.cost, human["section_%d"%row.id])
    
    # スキルの中心点Vecを求める
    skill_a = lpDot(human.skill_a, human["section_%d"%row.id]) / row.num
    skill_b = lpDot(human.skill_b, human["section_%d"%row.id]) / row.num
    skill_c = lpDot(human.skill_c, human["section_%d"%row.id]) / row.num
    skill_d = lpDot(human.skill_d, human["section_%d"%row.id]) / row.num
        
    # 中心点が近い
    cost -= skill_a * row.skill_center_a
    cost -= skill_b * row.skill_center_b
    cost -= skill_c * row.skill_center_c
    cost -= skill_d * row.skill_center_d
    
    # 分散が1=大きい -1=小さい
    # lpDot(human.skill_a, human["section_%d"%row.id]) - skill_a * row.num
    m+= -row.dist_v <=lpDot(human.skill_a, human["section_%d"%row.id]) - skill_a * row.num <= -.01
    m+=  .01 <= lpDot(human.skill_a, human["section_%d"%row.id]) - skill_a * row.num <= row.dist_v
    cost +=  row.dist_v
    
    
m+= cost

/Users/kasajei/.virtualenvs/mo/lib/python3.6/site-packages/pulp/pulp.py:1335: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [201]:
for index, row in section.iterrows():
    m += lpSum(human["section_%d"%row.id])==row.num

In [202]:
for index, row in human.iterrows():
    m += (row.section_1+row.section_2+row.section_3+row.section_4+row.section_5)==1

In [203]:
pulp.LpStatus[m.solve()]

'Infeasible'

In [193]:
print(m)

NoName:
MINIMIZE
-1*dist_0 + -1*dist_1 + -1*dist_2 + -1*dist_3 + -1*dist_4 + 4.8*v1_0 + 3*v1_1 + 4*v1_10 + 3*v1_11 + 2*v1_12 + 4*v1_13 + 6.8*v1_14 + 2.8*v1_15 + 2*v1_16 + 4.8*v1_17 + 6*v1_18 + 3.8*v1_19 + 4*v1_2 + 1.8*v1_3 + 5*v1_4 + 0.8*v1_5 + 2*v1_6 + 3*v1_7 + 6*v1_8 + 1.8*v1_9 + 4.8*v2_0 + 3*v2_1 + 3.8*v2_10 + 3*v2_11 + 2*v2_12 + 4*v2_13 + 6.8*v2_14 + 3*v2_15 + 2*v2_16 + 5*v2_17 + 5.8*v2_18 + 4*v2_19 + 3.8*v2_2 + 2*v2_3 + 5*v2_4 + 1*v2_5 + 2*v2_6 + 3*v2_7 + 5.8*v2_8 + 2*v2_9 + 5*v3_0 + 2.75*v3_1 + 4*v3_10 + 3*v3_11 + 2*v3_12 + 3.75*v3_13 + 7*v3_14 + 3*v3_15 + 1.75*v3_16 + 4.75*v3_17 + 6*v3_18 + 4*v3_19 + 4*v3_2 + 2*v3_3 + 4.75*v3_4 + 1*v3_5 + 1.75*v3_6 + 3*v3_7 + 5.75*v3_8 + 2*v3_9 + 5*v4_0 + 3*v4_1 + 4*v4_10 + 2.6666666666666665*v4_11 + 1.6666666666666667*v4_12 + 4*v4_13 + 6.666666666666667*v4_14 + 3*v4_15 + 2*v4_16 + 5*v4_17 + 5.666666666666667*v4_18 + 3.6666666666666665*v4_19 + 3.6666666666666665*v4_2 + 2*v4_3 + 4.666666666666667*v4_4 + 1*v4_5 + 2*v4_6 + 2.6666666666666665*v4_7 +

In [194]:
human['section_1_v'] = human.section_1.apply(value)
human['section_2_v'] = human.section_2.apply(value)
human['section_3_v'] = human.section_3.apply(value)
human['section_4_v'] = human.section_4.apply(value)
human['section_5_v'] = human.section_5.apply(value)
section["dist_v"] = section.dist_v.apply(value)

In [195]:
section

,id,num,skill_center_a,skill_center_b,skill_center_c,skill_center_d,dist,dist_v
0,1,5,1,0,0,0,1,0.0
1,2,5,0,1,0,0,-1,0.0
2,3,4,0,0,1,0,1,0.0
3,4,3,0,0,0,1,1,0.0
4,5,3,0,1,0,0,-1,0.0


In [196]:
human[human.section_1_v==1]

,id,cost,skill_a,skill_b,skill_c,skill_d,skill_a_uni,skill_b_uni,skill_c_uni,skill_d_uni,section_1,section_2,section_3,section_4,section_5,section_1_v,section_2_v,section_3_v,section_4_v,section_5_v
3,4,2,1,0,0,0,1.000000,0.0,0.0,0.000000,v1_3,v2_3,v3_3,v4_3,v5_3,1.0,0.0,0.0,0.0,0.0
5,6,1,1,0,0,0,1.000000,0.0,0.0,0.000000,v1_5,v2_5,v3_5,v4_5,v5_5,1.0,0.0,0.0,0.0,0.0
9,10,2,1,0,0,0,1.000000,0.0,0.0,0.000000,v1_9,v2_9,v3_9,v4_9,v5_9,1.0,0.0,0.0,0.0,0.0
15,16,3,1,0,0,0,1.000000,0.0,0.0,0.000000,v1_15,v2_15,v3_15,v4_15,v5_15,1.0,0.0,0.0,0.0,0.0
19,20,4,1,0,0,1,0.707107,0.0,0.0,0.707107,v1_19,v2_19,v3_19,v4_19,v5_19,1.0,0.0,0.0,0.0,0.0


In [182]:
human[human.section_2_v==1]

,id,cost,skill_a,skill_b,skill_c,skill_d,skill_a_uni,skill_b_uni,skill_c_uni,skill_d_uni,section_1,section_2,section_3,section_4,section_5,section_1_v,section_2_v,section_3_v,section_4_v,section_5_v
0,1,5,1,1,0,0,0.707107,0.707107,0.0,0.000000,v1_0,v2_0,v3_0,v4_0,v5_0,0.0,1.0,0.0,0.0,0.0
5,6,1,1,0,0,0,1.000000,0.000000,0.0,0.000000,v1_5,v2_5,v3_5,v4_5,v5_5,0.0,1.0,0.0,0.0,0.0
7,8,3,0,0,0,1,0.000000,0.000000,0.0,1.000000,v1_7,v2_7,v3_7,v4_7,v5_7,0.0,1.0,0.0,0.0,0.0
14,15,7,1,1,0,1,0.577350,0.577350,0.0,0.577350,v1_14,v2_14,v3_14,v4_14,v5_14,0.0,1.0,0.0,0.0,0.0
18,19,6,0,1,0,1,0.000000,0.707107,0.0,0.707107,v1_18,v2_18,v3_18,v4_18,v5_18,0.0,1.0,0.0,0.0,0.0


In [149]:
human[human.section_3_v==1]

,id,cost,skill_a,skill_b,skill_c,skill_d,skill_a_uni,skill_b_uni,skill_c_uni,skill_d_uni,section_1,section_2,section_3,section_4,section_5,section_1_v,section_2_v,section_3_v,section_4_v,section_5_v
1,2,3,0,0,1,0,0.0,0.0,1.000000,0.000000,v1_1,v2_1,v3_1,v4_1,v5_1,0.0,0.0,1.0,0.0,0.0
4,5,5,0,0,1,1,0.0,0.0,0.707107,0.707107,v1_4,v2_4,v3_4,v4_4,v5_4,0.0,0.0,1.0,0.0,0.0
6,7,2,0,0,1,0,0.0,0.0,1.000000,0.000000,v1_6,v2_6,v3_6,v4_6,v5_6,0.0,0.0,1.0,0.0,0.0
16,17,2,0,0,1,0,0.0,0.0,1.000000,0.000000,v1_16,v2_16,v3_16,v4_16,v5_16,0.0,0.0,1.0,0.0,0.0


In [150]:
human[human.section_4_v==1]

,id,cost,skill_a,skill_b,skill_c,skill_d,skill_a_uni,skill_b_uni,skill_c_uni,skill_d_uni,section_1,section_2,section_3,section_4,section_5,section_1_v,section_2_v,section_3_v,section_4_v,section_5_v
7,8,3,0,0,0,1,0.000000,0.0,0.0,1.000000,v1_7,v2_7,v3_7,v4_7,v5_7,0.0,0.0,0.0,1.0,0.0
12,13,2,0,0,0,1,0.000000,0.0,0.0,1.000000,v1_12,v2_12,v3_12,v4_12,v5_12,0.0,0.0,0.0,1.0,0.0
19,20,4,1,0,0,1,0.707107,0.0,0.0,0.707107,v1_19,v2_19,v3_19,v4_19,v5_19,0.0,0.0,0.0,1.0,0.0


In [151]:
human[human.section_5_v==1]

,id,cost,skill_a,skill_b,skill_c,skill_d,skill_a_uni,skill_b_uni,skill_c_uni,skill_d_uni,section_1,section_2,section_3,section_4,section_5,section_1_v,section_2_v,section_3_v,section_4_v,section_5_v
2,3,4,0,1,0,1,0.00000,0.707107,0.0,0.707107,v1_2,v2_2,v3_2,v4_2,v5_2,0.0,0.0,0.0,0.0,1.0
10,11,4,0,1,0,0,0.00000,1.000000,0.0,0.000000,v1_10,v2_10,v3_10,v4_10,v5_10,0.0,0.0,0.0,0.0,1.0
14,15,7,1,1,0,1,0.57735,0.577350,0.0,0.577350,v1_14,v2_14,v3_14,v4_14,v5_14,0.0,0.0,0.0,0.0,1.0
